<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/nbs/02b_misc_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#default_exp misc_utils

# Miscellaneous Utilities

In [ ]:
#hide
#colab
!pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 100kB/s 
     |████████████████████████████████| 61kB 3.3MB/s 


In [ ]:
#hide
#colab
# !pip install -Uqq git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastai --upgrade

     |████████████████████████████████| 194kB 6.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 


In [ ]:
#hide
#colab
!pip install -qqq nbdev

In [ ]:
#hide
#colab
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
#hide
!pip freeze | grep torch
!pip freeze | grep fast
!pip freeze | grep nbdev

torch==1.7.0+cu101
torch-xla==1.7
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101
fastai==2.2.5
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


## Patch the fastai `_BaseOptimizer` to be Pickable
Patch the `fastai.optimizer._BaseOptimizer` `__getstate__` and `__setstate__` methods which are used in pickling fastai optimizers. 

This should fix the bug where running the learner on multiple TPU cores on XLA triggers an error in which the method `_fetch_gradients(optimizer)` fails in the statement `for param_group in optimizer.__getstate__()['param_groups']:` in the  `torch_xla.core.xla_model` module. 

The patch modifies the copy constructor to include the param_groups and defaults.

In [ ]:
#hide
#colab
from nbdev.showdoc import *

In [ ]:
#export
from fastai.optimizer import _BaseOptimizer
from fastcore.basics import patch

@patch
def __getstate__(self:_BaseOptimizer):
    "Pickling opt state should include `param_groups` and `defaults` "
    d = {
            'state': self.state_dict(),
            'param_groups': self.param_groups,
        }
    if hasattr(self,'defaults'):
        d['defaults'] = self.defaults
    return d

@patch
def __setstate__(self:_BaseOptimizer, data):
    "Pickling opt state should include `param_groups` and `defaults` "

    if 'defaults' in data:
        self.defaults = data['defaults']
    self.load_state_dict(data['state'])
    self.param_groups = data['param_groups']

In [ ]:
#hide_input
#colab
show_doc(_BaseOptimizer.__getstate__)
show_doc(_BaseOptimizer.__setstate__)

In [ ]:
#hide
#TODO add tests for pickling optimizers